# Week 4: Wavelet Transformation for Feature Selection 
Shateesh Bhugwansing

In an attempt to increase our classification scores, I'm attempting to use Discrete Wavelet Transformation to decompose the EEG data. The resulting sub bands were ranked using their energy levels, and then fit into classifiers.

The following papers were used as a reference for this procedure: 
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5702353/
* https://link.springer.com/article/10.1007%2Fs13246-015-0333-x#Sec3

### PLEASE NOTE: 
* the python package 'pywt' (PyWavelet) and 'mne' must be installed in your environment in order to run this notebook. 
* the data sources are local to my machine/external hard drive. We haven't set up a cloud storage system yet.

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.model_selection import StratifiedKFold, ShuffleSplit, cross_val_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import metrics
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler


import warnings
warnings.simplefilter("ignore", DeprecationWarning)
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams.update({'font.size': 12})


#import mne to read EEG data 
import mne 
from mne.decoding import Vectorizer, CSP

#import pywavelet for DWT 
import pywt 